In [22]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
from dateutil.relativedelta import relativedelta
os.chdir("C:/Users/herna/OneDrive/Documentos/Labo_3/labo3_empresa3_repo/datasets")

In [25]:
#maestro de productos
#Lea el Dataset base
#data = "emp3_sellout_base_period_product.csv"
data = "emp3_sellout_lags_deltalags_product_categorias.csv" #FE que dejo el NB de EMi
df = pd.read_csv(data)

columnas_a_eliminar = ['periodo']
df = df.drop(columnas_a_eliminar, axis=1)
df = df.rename(columns={'periodo_fecha': 'periodo'})

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31243 entries, 0 to 31242
Columns: 300 entries, periodo to cat3_delta_lag_35
dtypes: bool(1), float64(292), int64(3), object(4)
memory usage: 71.3+ MB


**setting the parametros generales**

In [26]:
periods_to_train = ['2019-10-01','2019-09-01','2019-08-01','2019-07-01','2019-06-01','2019-05-01','2019-04-01','2019-03-01','2019-02-01','2019-01-01','2018-12-01','2018-11-01' ]
#periods_to_train = ['2019-10-01']

In [27]:
first_df = True
i = 1
for periodo in periods_to_train:
    
    #cada periodo
    print("Procesando periodo:", periodo)
    
    #tomo periodo para generar mi dataset de train
    df_train = df[df['periodo'] == periodo]
    
    #========================================================================================
    #genero el campo tn_2 target
    #========================================================================================
    fecha_2 = datetime.strptime(periodo, '%Y-%m-%d')  # Convierte la cadena a objeto datetime
    fecha_2 += relativedelta(months=2)  # Suma 2 meses a la fecha
    fecha_2 = fecha_2.strftime('%Y-%m-%d')
    print("periodo tn_2:", fecha_2)
    filtro = df['periodo'] == fecha_2
    df_filtrado = df[filtro]

    # Seleccionar solo las columnas 'product_id' y 'tn', y renombrar 'tn' como 'tn_2'
    df_nuevo = df_filtrado[['product_id', 'tn']].rename(columns={'tn': 'tn_mas_2'})
    df_train = pd.merge(df_train, df_nuevo, on='product_id', how='left')
    
    #========================================================================================
    #tomo rango de fechas para 13 lags campo lag tn
    #========================================================================================
    fecha = datetime.strptime(periodo, '%Y-%m-%d')  # Convierte la cadena a objeto datetime
    fecha += relativedelta(months=-13)  # Suma 2 meses a la fecha
    fecha_inicio = fecha.strftime('%Y-%m-%d')
    print("Fecha ini lags:", fecha.strftime('%Y-%m-%d'))
    
    fecha = datetime.strptime(periodo, '%Y-%m-%d')  # Convierte la cadena a objeto datetime
    fecha += relativedelta(months=-0)  # Suma 2 meses a la fecha
    fecha_fin = fecha.strftime('%Y-%m-%d')
    print("Fecha fin lags:", fecha.strftime('%Y-%m-%d'))
    fecha_inicio = pd.Timestamp(fecha_inicio)
    fecha_fin = pd.Timestamp(fecha_fin)

    lag = 13
    # Iterar a través de los periodos en el rango
    for fecha in pd.date_range(start=fecha_inicio, end=fecha_fin, freq='M'):
        #print('entro')
        # Obtiene el primer día del mes para cada fecha
        fecha = pd.to_datetime(fecha)
        primer_dia_del_mes = fecha - pd.DateOffset(days=fecha.day - 1)
        primer_dia_del_mes = primer_dia_del_mes.strftime('%Y-%m-%d')
        #print(primer_dia_del_mes)
        # Filtrar el DataFrame por el periodo actual
        filtro = df['periodo'] == primer_dia_del_mes
        df_filtrado = df[filtro]
        # Seleccionar solo las columnas 'product_id' y 'tn', y renombrar 'tn' como 'lag_tn_xx'
        df_nuevo = df_filtrado[['product_id', 'tn']].rename(columns={'tn': 'lag_tn_'+str(lag)})


        df_train = pd.merge(df_train, df_nuevo, on='product_id', how='left')
        lag = lag - 1

    #========================================================================================
    #tomo rango de fechas para 13 lags campo lag months
    #========================================================================================

    lag = 13
    # Iterar a través de los periodos en el rango
    for fecha in pd.date_range(start=fecha_inicio, end=fecha_fin, freq='M'):
        # Obtiene el primer día del mes para cada fecha
        fecha = pd.to_datetime(fecha)
        primer_dia_del_mes = fecha - pd.DateOffset(days=fecha.day - 1)
        primer_dia_del_mes = primer_dia_del_mes.strftime('%Y-%m-%d')
        #print(primer_dia_del_mes)
        # Filtrar el DataFrame por el periodo actual
        filtro = df['periodo'] == primer_dia_del_mes
        df_filtrado = df[filtro]
        #print(str(fecha.month))
        df_train['lag_month_'+str(lag)] = str(fecha.month)
        lag = lag - 1

    #========================================================================================
    #genero el campo lag_tn_mean_2_3_6_12 
    #========================================================================================
    mean_periods = [-2,-3,-6,-12]
    fecha_from = datetime.strptime(periodo, '%Y-%m-%d')  # Convierte la cadena a objeto datetime
    fecha_from += relativedelta(months=-1)  # Suma 2 meses a la fecha
    fecha_from = fecha_from.strftime('%Y-%m-%d')
    print("mean from:", fecha_from)
    
    for mean_period in mean_periods:
        fecha_to = datetime.strptime(periodo, '%Y-%m-%d')  # Convierte la cadena a objeto datetime
        fecha_to += relativedelta(months=mean_period)  # Suma 2 meses a la fecha
        fecha_to = fecha_to.strftime('%Y-%m-%d')
        print("mean to "+str(mean_period) , fecha_to)
        filtro = (df['periodo'] >= fecha_to) & (df['periodo'] <= fecha_from)
        df_filtrado = df[filtro]
        df_filtrado = df_filtrado.groupby(['product_id']).agg({'tn': 'mean'}).reset_index()
        df_filtrado = df_filtrado.rename(columns={'tn': 'lag_tn_mean_'+str(abs(mean_period))})
        df_train = pd.merge(df_train, df_filtrado, on='product_id', how='left')

    #========================================================================================
    #genero el campo lag_tn_sum_q1_q2_q3_q4 y lag_tn_mean_q1_q2_q3_q4
    #========================================================================================
    quarters = [[-1,-3], [-4,-6], [-7,-9], [-10,-12]]
    quarters_name = ['q1', 'q2', 'q3', 'q4']
    
    for i in range(len(quarters)):
        
        fecha_from = datetime.strptime(periodo, '%Y-%m-%d')  # Convierte la cadena a objeto datetime
        fecha_from += relativedelta(months=quarters[i][0])  # Suma 2 meses a la fecha
        fecha_from = fecha_from.strftime('%Y-%m-%d')
        print("q from:", fecha_from)

        fecha_to = datetime.strptime(periodo, '%Y-%m-%d')  # Convierte la cadena a objeto datetime
        fecha_to += relativedelta(months=quarters[i][1])  # Suma 2 meses a la fecha
        fecha_to = fecha_to.strftime('%Y-%m-%d')
        print("q to:", fecha_to)
        
        filtro = (df['periodo'] >= fecha_to) & (df['periodo'] <= fecha_from)
        df_filtrado = df[filtro]
        #para sum
        df_filtrado_sum = df_filtrado.groupby(['product_id']).agg({'tn': 'sum'}).reset_index()
        df_filtrado_sum = df_filtrado_sum.rename(columns={'tn': 'lag_sum_tn_'+quarters_name[i]})
        df_train = pd.merge(df_train, df_filtrado_sum, on='product_id', how='left')
        #para mean
        df_filtrado_mean = df_filtrado.groupby(['product_id']).agg({'tn': 'mean'}).reset_index()
        df_filtrado_mean = df_filtrado_mean.rename(columns={'tn': 'lag_mean_tn_'+quarters_name[i]})
        df_train = pd.merge(df_train, df_filtrado_mean, on='product_id', how='left')    

        
    if first_df:
        first_df = False
        df_train_final = df_train
    else:
        df_train_final = pd.concat([df_train_final, df_train], axis=0, ignore_index=True)
        

Procesando periodo: 2019-10-01
periodo tn_2: 2019-12-01
Fecha ini lags: 2018-09-01
Fecha fin lags: 2019-10-01
mean from: 2019-09-01
mean to -2 2019-08-01
mean to -3 2019-07-01
mean to -6 2019-04-01
mean to -12 2018-10-01
q from: 2019-09-01
q to: 2019-07-01
q from: 2019-06-01
q to: 2019-04-01
q from: 2019-03-01
q to: 2019-01-01
q from: 2018-12-01
q to: 2018-10-01
Procesando periodo: 2019-09-01
periodo tn_2: 2019-11-01
Fecha ini lags: 2018-08-01
Fecha fin lags: 2019-09-01
mean from: 2019-08-01
mean to -2 2019-07-01
mean to -3 2019-06-01
mean to -6 2019-03-01
mean to -12 2018-09-01
q from: 2019-08-01
q to: 2019-06-01
q from: 2019-05-01
q to: 2019-03-01
q from: 2019-02-01
q to: 2018-12-01
q from: 2018-11-01
q to: 2018-09-01
Procesando periodo: 2019-08-01
periodo tn_2: 2019-10-01
Fecha ini lags: 2018-07-01
Fecha fin lags: 2019-08-01
mean from: 2019-07-01
mean to -2 2019-06-01
mean to -3 2019-05-01
mean to -6 2019-02-01
mean to -12 2018-08-01
q from: 2019-07-01
q to: 2019-05-01
q from: 2019-

In [28]:
#========================================================================================
#genero el campo lag_trend_q1 q2 q3 q4
#========================================================================================

def calculate_trend(row, *lag_column_names):
    # Filtrar valores no nulos para el cálculo de tendencia
    lag_values = [row[col] for col in lag_column_names if not np.isnan(row[col])]

    # Verificar que haya suficiente variación en los datos para un ajuste lineal
    if len(set(lag_values)) > 1:
        coefficients = np.polyfit([1, 2, 3], lag_values, 1)
        return coefficients[0]
    else:
        return 0.0

print("Columns Trend by quarters")
#Q1
column_names = ["lag_tn_1", "lag_tn_2", "lag_tn_3"]
df_train_final[column_names] = df_train_final[column_names].fillna(0)
df_train_final["lag_trend_q1"] = df_train_final.apply(calculate_trend, args=column_names, axis=1).copy()
#Q2
column_names = ["lag_tn_4", "lag_tn_5", "lag_tn_6"]
df_train_final[column_names] = df_train_final[column_names].fillna(0)
df_train_final["lag_trend_q2"] = df_train_final.apply(calculate_trend, args=column_names, axis=1).copy()
#Q3
column_names = ["lag_tn_7", "lag_tn_8", "lag_tn_9"]
df_train_final[column_names] = df_train_final[column_names].fillna(0)
df_train_final["lag_trend_q3"] = df_train_final.apply(calculate_trend, args=column_names, axis=1).copy()
#Q4
column_names = ["lag_tn_10", "lag_tn_11", "lag_tn_12"]
df_train_final[column_names] = df_train_final[column_names].fillna(0)
df_train_final["lag_trend_q4"] = df_train_final.apply(calculate_trend, args=column_names, axis=1).copy()

Columns Trend by quarters


In [29]:
df_train_final.head()

,periodo,product_id,tn,cust_request_qty,cust_request_tn,plan_precios_cuidados,cat1,cat2,cat3,sku_size,...,lag_sum_tn_q2,lag_mean_tn_q2,lag_sum_tn_q3,lag_mean_tn_q3,lag_sum_tn_q4,lag_mean_tn_q4,lag_trend_q1,lag_trend_q2,lag_trend_q3,lag_trend_q4
0,2019-10-01,20001,1561.50552,367,1587.87525,0,HC,ROPA LAVADO,Liquido,3000.0,...,4387.35850,1462.452833,4005.52367,1335.174557,5594.90012,1864.966707,9.493785,268.850395,-97.441510,404.255815
1,2019-10-01,20002,1979.53635,312,2013.36305,0,HC,ROPA LAVADO,Liquido,3000.0,...,3250.97704,1083.659013,3393.42652,1131.142173,4154.75558,1384.918527,-11.868860,179.629575,91.580995,184.517870
2,2019-10-01,20003,1081.36645,404,1091.94793,0,FOODS,ADEREZOS,Mayonesa,475.0,...,1817.84943,605.949810,2361.13586,787.045287,3290.08853,1096.696177,-126.284010,-48.524400,163.364545,271.756710
3,2019-10-01,20004,1064.69633,508,1078.32756,0,FOODS,ADEREZOS,Mayonesa,240.0,...,1737.21393,579.071310,1572.81129,524.270430,2197.58232,732.527440,-132.228105,-100.242550,-54.216855,112.053045
4,2019-10-01,20005,996.78275,418,1040.72694,0,FOODS,ADEREZOS,Mayonesa,120.0,...,2398.65873,799.552910,1261.69775,420.565917,1735.63858,578.546193,-66.889150,-125.699080,-62.314745,260.553290


In [30]:
# Exportar el DataFrame a un archivo CSV
df_train_final.to_csv(r'C:\Users\herna\OneDrive\Documentos\Labo_3\labo3_empresa3_repo\datasets\emp3_sellout_base_period_product_FE.csv', index=False)